# Acquire

The acquire module is how data is loaded into datachef.

The acquire module uses dot notation to specify format and origin (local vs http etc).

Simple examples for common tabular data formats follow.




## CSV Data From Local

Creating a single selectable table-like object from a local csv file.

In [39]:
from datachef import acquire, preview

# Argument is the location of any csv file on your machine
# This can be a string or a python Path object.
table = acquire.csv.local("../../tests/fixtures/csv/bands-wide.csv")
preview(table)

,A,B,C,D,E,F,G,H,I,J,K
1,,,,,,,,,,,
2,,,Houses,Cars,Boats,,,,Houses,Cars,Boats
3,Beatles,,,,,,Rolling Stones,,,,
4,,John,1,5,9,,,Keith,2,6,10
5,,Paul,2,6,10,,,Mick,3,7,11
6,,George,2,7,11,,,Charlie,3,8,12
7,,Ringo,4,8,12,,,Ronnie,5,9,13
8,,,,,,,,,,,


## CSV Data from Http(s)

You can also load a csv via http as per the following example:

In [40]:
from datachef import acquire, preview

# Argument is any csv file accessible via http or https
table = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")
preview(table)

,A,B,C,D,E,F,G,H,I,J,K
1,,,,,,,,,,,
2,,,Houses,Cars,Boats,,,,Houses,Cars,Boats
3,Beatles,,,,,,Rolling Stones,,,,
4,,John,1,5,9,,,Keith,2,6,10
5,,Paul,2,6,10,,,Mick,3,7,11
6,,George,2,7,11,,,Charlie,3,8,12
7,,Ringo,4,8,12,,,Ronnie,5,9,13
8,,,,,,,,,,,


## Customising Csv Loads

Both the local and http csv loaders are wrappers around [the csv reader from the standard python library csv package](https://docs.python.org/3/library/csv.html) and propogate keyword arguments.

This means you can pass any keyword arguments through to `acquire.csv.local()` and `acquire.csv.http()` that you could pass to the `csv.reader()` method.

As an example here we are loading a csv file using a `|` delimiter in place of commas. [This is the example data we're using](https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/pipe-delimited.csv).

In [41]:
from datachef import acquire, preview

# Lets specify a different delimiter
table = acquire.csv.local("../../tests/fixtures/csv/pipe-delimited.csv", delimiter="|")
preview(table)

,A,B,C
1,Age,Male,Female
2,1,3,9
3,14,54,12
4,9,0,3


# Loading Xlsx Data

## Xlsx Data From Local

## Xlsx Data From Https(s)

# Loading Xls Data

## Xls Data From Local

## Xlsx Data From Http(s)